In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os

In [ ]:
# load age key
# root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
# root = "/net/trapnell/vol1/home/nlammers/projects/data/morphseq/"
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
# root = "/Users/nick/Desktop/"

metadata_path = os.path.join(root, "metadata", "combined_metadata_files")
embryo_df = pd.read_csv(os.path.join(metadata_path, "embryo_metadata_df02.csv"))

# remove rows and add variables
morph_experiments = []#"20230830", "20230831", "20231207", "20231208"]
ref_experiments = ["20240626"]
# generate an absolute referemce
# eid_vec = np.asarray([snip[:12] for snip in age_key_df["snip_id"]])
# date_vec = np.asarray([snip[:8] for snip in age_key_df["snip_id"]])
# age_key_df["embryo_id"] = eid_vec
# age_key_df["experiment_date"] = date_vec
eid_vec = embryo_df["embryo_id"].to_numpy()
eid_index = np.unique(eid_vec)
embryo_df["abs_stage_hpf"] = np.nan

for eid in eid_index:
    abs_time_vec = embryo_df.loc[eid_vec==eid, "Time Rel (s)"].to_numpy() / 3600
    snip_id0 = embryo_df.loc[eid_vec==eid, "snip_id"].to_numpy()[0]
    time_int = int(snip_id0[-4:])
    if time_int > 1:
        abs_time_vec = abs_time_vec - abs_time_vec[0]
    pd_stage_vec = embryo_df.loc[eid_vec==eid, "inferred_stage_hpf"].to_numpy()
    embryo_df.loc[eid_vec==eid, "abs_stage_hpf"] = pd_stage_vec[0] + abs_time_vec

    
embryo_df["experiment_date"] = embryo_df["experiment_date"].astype(str)

embryo_df = embryo_df.loc[embryo_df["use_embryo_flag"]==1, :]
# ref_flags = np.asarray([True if embryo_df.loc[i, "experiment_date"] in ref_experiments
#                          else False for i in embryo_df.index])


### Plot inferred stage vs. prediction using simple linear formula

In [ ]:
fig = px.scatter(embryo_df, x="predicted_stage_hpf", y="inferred_stage_hpf",opacity=0.25,
                 color="experiment_date", template="plotly")

fig.update(layout_yaxis_range = [0,80])
fig.update(layout_xaxis_range = [0,80])

fig.update_scenes(aspectmode='data')
fig.show()

In [ ]:
# import pandas as pd
# import plotly.express as px
# import os 
# import numpy as np

# root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
# metadata_path = os.path.join(root, "metadata", "combined_metadata_files")
                             
# embryo_df = pd.read_csv(os.path.join(metadata_path, "embryo_metadata_df.csv"), index_col=0)
# embryo_df.head()

In [ ]:
# old_df = pd.read_csv(os.path.join(metadata_path, "_Archive", "embryo_metadata_df01.csv"))
# old_df.head()

In [ ]:
# old_merge_skel = old_df.loc[:, ["snip_id", "out_of_frame_flag"]].drop_duplicates(subset="snip_id")
# print(old_merge_skel.shape)

# embryo_df_merge = embryo_df.drop_duplicates(subset="snip_id")
# print(embryo_df.shape)

# embryo_df01 = embryo_df_merge.merge(old_merge_skel, how="left", on="snip_id", indicator=True)
# print(np.mean(embryo_df01["_merge"]=="both"))
# embryo_df01 = embryo_df01.drop(labels=["_merge"], axis=1)
# embryo_df01.head()

In [ ]:
# embryo_df01.to_csv(os.path.join(metadata_path, "embryo_metadata_df01.csv"), index=False)

In [ ]:
# embryo_df_merge.to_csv(os.path.join(metadata_path, "embryo_metadata_df.csv"), index=False)

In [ ]:
fig = px.scatter(embryo_df.loc[embryo_df["experiment_date"]=="20231110", :], x="predicted_stage_hpf", y="length_um",opacity=0.25,
                 color="experiment_date", template="plotly")
fig.show()

In [ ]:
stage_df = pd.read_csv(os.path.join(root, "stage_ref_df.csv"), index_col=0)
stage_df.head()

px.scatter(stage_df, x="stage_hpf", y="sa_um")

### Plot inferred stage vs. real time

In [ ]:
fig = px.scatter(age_key_df, x="abs_stage_hpf", y="inferred_stage_hpf_reg",opacity=0.25,
                 color="experiment_date", template="plotly")
fig.show()

In [ ]:
fig = px.scatter(age_key_df.loc[age_key_df["experiment_date"].astype(str)=="20240509", :], 
                 x="calc_stage_hpf", y="inferred_stage_hpf_reg",opacity=0.75,
                 color="short_pert_name", template="plotly", hover_data={"embryo_id"})
fig.show()

In [ ]:
dispersion = age_key_df.loc[age_key_df["experiment_date"].astype(str)=="20231206", :].groupby(
                'embryo_id')[['calc_stage_hpf', 'abs_stage_hpf']].agg(lambda x: x.max() - x.min()).reset_index()

In [ ]:
px.scatter(dispersion, x="abs_stage_hpf", y="calc_stage_hpf")

In [ ]:
start_df = age_key_df.loc[age_key_df["experiment_date"].astype(str)=="20231206", :].groupby(
                'embryo_id')[['calc_stage_hpf', 'abs_stage_hpf']].agg(['max', 'min']).reset_index()

start_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in start_df.columns]
start_df.head()

In [ ]:
px.scatter(start_df, x="calc_stage_hpf_min", y="calc_stage_hpf_max")

In [ ]:
px.scatter(start_df, x="abs_stage_hpf_min", y="abs_stage_hpf_max")

#### That is super weird. Let's check the embryo df to see if the issue is there as well

In [ ]:
data_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/training_data/20241008/VAE_z100_ne250_base_model/VAE_training_2024-10-08_21-40-44/figures/embryo_stats_df.csv"
embryo_df = pd.read_csv(data_path, index_col=0)
embryo_df.head()

metadata_df = pd.read_csv("/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/training_data/20241008/embryo_metadata_df_train.csv")
metadata_df = metadata_df.loc[:, ["snip_id", "temperature", "embryo_id", "Time Rel (s)"]]

# add temperature and 
embryo_df = embryo_df.merge(metadata_df, on="snip_id", how="left")
embryo_df.reset_index(inplace=True, drop=True)

embryo_df.head()

In [ ]:
embryo_df = embryo_df.rename(columns={"Time Rel (s)":"time_rel"})
embryo_df["time_rel"] = embryo_df["time_rel"] / 3600
start_df2 = embryo_df.loc[embryo_df["experiment_date"].astype(str)=="20231206", :].groupby(
                'embryo_id')[['predicted_stage_hpf', 'time_rel']].agg(['max', 'min']).reset_index()

start_df2.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in start_df2.columns]
start_df2.head()

In [ ]:
px.scatter(start_df2, x="predicted_stage_hpf_min", y="time_rel_max")

In [ ]:
px.scatter(age_key_df.loc[age_key_df["experiment_date"]=="20231206", :], x="calc_stage_hpf", y="abs_time_hr")